In [73]:
import pulp
import numpy as np
import matplotlib_inline as plt

In [74]:
prob = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob2 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)
prob3 = pulp.LpProblem("Minimize_Cost", pulp.LpMinimize)

In [75]:
import numpy as np

def generate_random_directed_graph(num_nodes, num_edges, cost_range=(1, 10)):
    # check the number of nodes and edges
    if num_edges < num_nodes or num_edges > num_nodes * (num_nodes - 1) // 2:
        raise ValueError("Number of edges must be at least num_nodes and no more than num_nodes * (num_nodes - 1) // 2.")

    # product nodes
    nodes = [chr(i) for i in range(97, 97 + num_nodes)]

    # create a cycle to ensure the strong connection
    cycle_edges = [(nodes[i], nodes[(i + 1) % num_nodes]) for i in range(num_nodes)]
    
    # produce other possible edge
    possible_edges = [(nodes[i], nodes[j]) for i in range(num_nodes) for j in range(num_nodes) if i != j and (nodes[j], nodes[i]) not in cycle_edges and (nodes[i], nodes[j]) not in cycle_edges]

    # remove the duplicated edge and self cycle
    selected_edges = cycle_edges
    remaining_edges_count = num_edges - len(cycle_edges)
    while remaining_edges_count > 0:
        np.random.shuffle(possible_edges)
        new_edge = possible_edges[0]
        if new_edge not in selected_edges and (new_edge[1], new_edge[0]) not in selected_edges:
            selected_edges = selected_edges + [new_edge]
            remaining_edges_count -= 1
    
    # produce edges with costs
    edges_with_costs = {edge: np.random.randint(cost_range[0], cost_range[1] + 1) for edge in selected_edges}

    return nodes, edges_with_costs

# 使用该函数生成一个随机有向图
num_nodes = 10# 节点数量
num_edges = 45# 总边数，至少等于节点数
nodes, edges_with_costs = generate_random_directed_graph(num_nodes, num_edges)
for keys in edges_with_costs:
    edges_with_costs[keys] = float(edges_with_costs[keys]/100)

print("Nodes:", nodes)
print("Directed edges with costs:", edges_with_costs)

Nodes: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j']
Directed edges with costs: {('a', 'b'): 0.04, ('b', 'c'): 0.03, ('c', 'd'): 0.07, ('d', 'e'): 0.01, ('e', 'f'): 0.05, ('f', 'g'): 0.08, ('g', 'h'): 0.01, ('h', 'i'): 0.08, ('i', 'j'): 0.03, ('j', 'a'): 0.01, ('d', 'b'): 0.02, ('f', 'h'): 0.08, ('e', 'a'): 0.02, ('g', 'j'): 0.02, ('c', 'h'): 0.06, ('a', 'g'): 0.07, ('d', 'f'): 0.03, ('e', 'g'): 0.08, ('h', 'a'): 0.07, ('h', 'b'): 0.03, ('j', 'e'): 0.07, ('g', 'b'): 0.05, ('i', 'b'): 0.05, ('g', 'c'): 0.08, ('h', 'e'): 0.1, ('j', 'c'): 0.1, ('d', 'i'): 0.01, ('e', 'c'): 0.04, ('g', 'd'): 0.04, ('f', 'a'): 0.09, ('a', 'c'): 0.08, ('i', 'g'): 0.06, ('h', 'd'): 0.06, ('i', 'e'): 0.05, ('f', 'i'): 0.1, ('j', 'd'): 0.02, ('c', 'i'): 0.09, ('f', 'j'): 0.08, ('j', 'h'): 0.03, ('j', 'b'): 0.08, ('e', 'b'): 0.05, ('b', 'f'): 0.03, ('d', 'a'): 0.06, ('a', 'i'): 0.05, ('c', 'f'): 0.04}


In [76]:
sp = input('Please select the root: ')
sp

'a'

In [77]:
import pulp
x = pulp.LpVariable.dicts('x',(nodes,nodes),0,1,cat=pulp.LpInteger)
y = pulp.LpVariable.dicts('y',(nodes,nodes),0,1,cat=pulp.LpInteger)
z = pulp.LpVariable.dicts('z',(nodes,nodes),0,1,cat=pulp.LpInteger)

In [78]:
prob +=pulp.lpSum(edges_with_costs[u,v] * x[u][v] for u, v in edges_with_costs)
prob2 +=pulp.lpSum(edges_with_costs[u,v] * y[u][v] for u, v in edges_with_costs)
prob3 +=pulp.lpSum(edges_with_costs[u,v] * z[u][v] for u, v in edges_with_costs)

In [79]:
for v in nodes:
    if v != sp:
        prob +=pulp.lpSum(x[u][v] for u in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for u in nodes if (u,v) in edges_with_costs) >= 1

for u in nodes:
    if u != sp:
        prob2 +=pulp.lpSum(y[u][v] for v in nodes if (u,v) in edges_with_costs) == 1
    prob3 +=pulp.lpSum(z[u][v] for v in nodes if (u,v) in edges_with_costs) >= 1

In [80]:
prob += pulp.lpSum(x[u][sp] for u in nodes if (u,sp) in edges_with_costs) == 0
prob += pulp.lpSum(x[sp][v] for v in nodes if (sp,v) in edges_with_costs)>= 1

prob2 += pulp.lpSum(y[u][sp] for u in nodes if (u,sp) in edges_with_costs) >=1
prob2 += pulp.lpSum(y[sp][v] for v in nodes if (sp,v) in edges_with_costs) ==0

In [81]:
from pulp import *
n = len(nodes)
subsets = [tuple(c) for c in allcombinations(nodes,n-1)]
subset = []
for s in subsets:
    if sp in s:
        subset.append(s)

In [82]:
for s in subset:
    sc = list(set(nodes)-set(s)) # complement of subset s
    summation = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation += x[u][v]
    prob += summation >= 1
    summation2 = 0.0
    for u in sc:
        for v in s:
            if (u, v) in edges_with_costs: # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation2 += y[u][v]
    prob2 += summation2 >= 1
    
for s in subsets:
    sc = list(set(nodes) - set(s))  # complement of subset s
    summation3 = 0.0
    for u in s:
        for v in sc:
            if (u, v) in edges_with_costs:  # Check if the (u, v) tuple is a valid key in edges_with_costs
                summation3 += z[u][v]
    prob3 += summation3 >= 1  

In [83]:
prob.solve()
print ("Status:", LpStatus[prob.status])
prob2.solve()
print ("Status:", LpStatus[prob2.status])
prob3.solve()
print ("Status:", LpStatus[prob3.status])

Status: Optimal
Status: Optimal
Status: Optimal


In [84]:
print ("Optimal Solution for out")
resultout = []
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			resultout.append((u,v))
resultout

Optimal Solution for out


[('a', 'g'),
 ('b', 'c'),
 ('d', 'b'),
 ('d', 'e'),
 ('d', 'f'),
 ('d', 'i'),
 ('g', 'h'),
 ('g', 'j'),
 ('j', 'd')]

In [85]:
costout = 0.0
for e in resultout:
    costout+= edges_with_costs[e]
costout

0.22

In [86]:
print ("Optimal Solution for in")
resultin = []
for u in nodes:
	for v in nodes:
		if y[u][v].value() == 1:
		    resultin.append((u,v))
resultin

Optimal Solution for in


[('b', 'f'),
 ('c', 'f'),
 ('d', 'e'),
 ('e', 'a'),
 ('f', 'j'),
 ('g', 'h'),
 ('h', 'b'),
 ('i', 'j'),
 ('j', 'a')]

In [87]:
costin = 0.0
for e in resultin:
    costin+= edges_with_costs[e]
costin

0.26

In [88]:
print ("Optimal Solution for strongly connected subgraph")
resultall = []
for u in nodes:
	for v in nodes:
		if z[u][v].value() == 1:
			resultall.append((u,v))
resultall

Optimal Solution for strongly connected subgraph


[('a', 'b'),
 ('b', 'c'),
 ('c', 'f'),
 ('d', 'e'),
 ('d', 'i'),
 ('e', 'a'),
 ('f', 'g'),
 ('g', 'h'),
 ('h', 'd'),
 ('i', 'j'),
 ('j', 'a')]

In [89]:
costall = 0.0
for e in resultall:
    costall+= edges_with_costs[e]
costall

0.33999999999999997

In [90]:
print ("Combine solution for out and in")
result=[]
for u in nodes:
	for v in nodes:
		if x[u][v].value() == 1:
			result.append((u,v))
		elif y[u][v].value() == 1:
			result.append((u,v))
            
result

Combine solution for out and in


[('a', 'g'),
 ('b', 'c'),
 ('b', 'f'),
 ('c', 'f'),
 ('d', 'b'),
 ('d', 'e'),
 ('d', 'f'),
 ('d', 'i'),
 ('e', 'a'),
 ('f', 'j'),
 ('g', 'h'),
 ('g', 'j'),
 ('h', 'b'),
 ('i', 'j'),
 ('j', 'a'),
 ('j', 'd')]

In [91]:
costcom = 0.0
for e in result:
    costcom+= edges_with_costs[e]
costcom

0.4600000000000001

In [92]:
print("Compare solution combine of out/in and for strongly connected subgraph")
print("result for strongly connected subgraph",resultall)
print("result for combine of out/in:", result)
print("Two result are same:", resultall==result)

Compare solution combine of out/in and for strongly connected subgraph
result for strongly connected subgraph [('a', 'b'), ('b', 'c'), ('c', 'f'), ('d', 'e'), ('d', 'i'), ('e', 'a'), ('f', 'g'), ('g', 'h'), ('h', 'd'), ('i', 'j'), ('j', 'a')]
result for combine of out/in: [('a', 'g'), ('b', 'c'), ('b', 'f'), ('c', 'f'), ('d', 'b'), ('d', 'e'), ('d', 'f'), ('d', 'i'), ('e', 'a'), ('f', 'j'), ('g', 'h'), ('g', 'j'), ('h', 'b'), ('i', 'j'), ('j', 'a'), ('j', 'd')]
Two result are same: False
